* By: Illya Barziy
* Email: illyabarziy@gmail.com
* Reference: __A Robust Estimator of the Efficient Frontier__ _by_ Marcos Lopez de Prado

## The Nested Clustered Optimization algorithm (NCO)

This description and the realization of the algorithm is based on the paper by _Marcos Lopez de Prado_ __A Robust Estimator of the Efficient Frontier__  [available here](https://papers.ssrn.com/abstract_id=3469961).

## Introduction

Convex optimization solutions in the problems of portfolio optimization tend to be unstable. The instability comes from two sources:
- noise in the input variables
- signal structure that magnifies the estimation errors in the input variables

The NCO algorithm tackles both sources of instability.

### Problem Statement

Say, we have a system with $N$ random variables, the expected value of draws of the variables are $\mu$ and the covariance matrix is $V$. We want to compute the vector $w$ that minimizes the variance of the system, where the variance is $w'Vw$, subject to $w'a$. Here $a$ characterizes the optimal solution. 

So, the problem is:

$min_{w}\frac{1}{2}w'Vw$

$s.t.: w'a = 1 $

In financial application, the optimal allocation $w^*$ known as the maximum Sharpe ratio, when $a = \mu$ (as it maximizes $\frac{w'\mu}{\sqrt{w'Vw}}$).

The optimal allocation $w^*$ known as the minimum variance portfolio, when $a = 1_{N}$, where $1_{N}$ is the vector of ones of size $N$ (as it maximizes the $w'Vw$).


Using the convex optimization methods, the optimal solution to the problem is

$w^* = \frac{V^{-1}a}{a'V^{-1}a}$

This is the Convex Optimization Solution (CVO).

As the input varaibles $V$ and $a$ are typically unknown, the estimations are used, which leads to unstable solutions where a small change of inputs will cause bix changes of $w^*$.


### Instability caused by noise

We have a matrix of iid random observations $X$, of size $TxN$. The underlying process is generating the observations with zero mean and $\sigma^2$ variance. 

The matrix $C = T^{-1}X'X$ has eigenvalues $\lambda$ that asymptotically converge (as $N -> +\infty$ and $T -> +\infty$ with $1 < \frac{T}{N} < +\infty$) to the Marcenko-Pastur probability density function.

$ f[\lambda] =
  \begin{cases}
    \frac{T}{N}\frac{\sqrt{(\lambda_+-\lambda)(\lambda-\lambda_-)}}{2\pi\lambda\sigma^2} & \quad \text{if } i \in [\lambda_-,\lambda_+] \text{,}\\
    0  & \quad \text{if } i \notin [\lambda_-,\lambda_+] \text{.}
  \end{cases}$

Where the maximim expected eigenvalue is 

$\lambda_+ = \sigma^2(1 + \sqrt{\frac{N}{T}})^2$

and the minimum expected eigenvalue is 

$\lambda_- = \sigma^2(1 - \sqrt{\frac{N}{T}})^2$

When $\sigma^2 = 1$, then $C$ is the correlation matrix associated with X.

The Marcenko-Pastur distribution explains why empirical covariance matrixces contain substantial amounts of noise. 

In many practical applications, $\frac{N}{T}->1$, thus the covariance's eigenvalues span a wide range $[\lambda_-,\lambda_+]$. Because $\lambda_- -> 0$ as $\frac{N}{T}->1$, the determinant of $V$ approaches zero and $V^{-1}$ cannot be estimated robustly, this is the source of unstable solutions $w^*$

The process of de-noising the covariance matrix that is used in the code examples in the end is described in a paper by _Potter M._, _J.P. Bouchaud_, _L. Laloux_ __“Financial applications of random matrix theory:
Old laces and new pieces.”__  [available here](https://arxiv.org/abs/physics/0507111).

### Instability caused by signal

Certain covariance structures can make the optimal solution unstable. Looking at the correlation matrix between two variables:

$C = \begin{equation}
  \begin{bmatrix}
    1 & \rho \\
    \rho & 1 
  \end{bmatrix}
\end{equation}$

Where the $\rho$ is the correlation between their outcomes. Matrix $C$ can be diagonalized as $CW = W\Lambda$, where

$\Lambda = \begin{equation}
  \begin{bmatrix}
    1 & +\rho \\
    1 & -\rho 
  \end{bmatrix}
\end{equation}$

$W = \begin{equation}
  \begin{bmatrix}
    \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
    \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} 
  \end{bmatrix}
\end{equation}$

The inverse of $C$ is

$C^{-1}=W\Lambda^{-1}W'=\frac{1}{|C|} = \begin{equation}
  \begin{bmatrix}
    1 & -\rho \\
    -\rho & 1 
  \end{bmatrix}
\end{equation}$

Where $|C|$ is the determinant of $C$, $|C|=(1+\rho)(1-\rho)=1-\rho^2$.

We can see that more $\rho$ derives from zero, the bigger one eigenvalue becomes relative to the other, causing the determinant of $C$ to approach zero, which makes the values of $C^{-1}$ explode.

When the correlation matrix is an identity matrix, the eigenvalue function is a horizontal line. Outside that ideal case, at least one subset of variables exhibits greater correlation among themselves than to the rest, forming a cluster within the correlation matrix. 

When $K$ variables form a cluster, they are more heavily exposed to a common eigenvector, which implies that the associated eigenvalue explains a greater amount of variance. 

But because the trace of the correlation matrix is exactly $N$, that means that an eigenvalue can only increase at the expense of the other $K-1$ eigenvalues in that cluster, resulting in a condition number greater than 1. Consequently, the greater the intra-cluster correlation is, the higher the condition number becomes. 

### De-noising function

The function provided below for de-noising the covariance works as follows:
- The given covariance matrix is transformed to the correlation matrix.
- The eigenvalues and eigenvectors of the correlation matrix are calculated.
- Using the Kernel Density Estimate algorithm a kernel of the eigenvalues is estimated.
- The Marcenko-Pastur pdf is fitted to the KDE estimate using the variance as the parameter for the optimization.
- From the obtained Marcenko-Pastur distribuiton, the maximum theoretical eigenvalue is calculated using the formula from "Instability caused by noise" part.
- The eigenvalues that are observed in the set above the theoretical value are all set to the average value.
- The new set of eigenvalues with the set of eigenvectors is used to obtain the new de-noised correlation matrix.
- The new correlation matrix is then transformed back to the new de-noised covariance matrix.

This is how the de-noising function can be used:

In [2]:
import mlfinlab as ml
import numpy as np

In [3]:
# A class that has the de-noising function
risk_estimators = ml.portfolio_optimization.RiskEstimators()

# Covariance matrix to de-noise
cov_matrix = np.array([[0.01, 0.002, -0.001],
                       [0.002, 0.04, -0.006],
                       [-0.001, -0.006, 0.01]])

# Relation of number of observations T to the number of variables N
tn_relation = 50

# The bandwidth of the KDE kernel
kde_bwidth = 0.25

# Finding the de-noised covariance matrix
cov_matrix_denoised = risk_estimators.de_noised_cov(cov_matrix, tn_relation, kde_bwidth)

# Outputting the result
print('The de-noised covariance matrix is:')
print(cov_matrix_denoised)

The de-noised covariance matrix is:
[[ 0.01        0.00267029 -0.00133514]
 [ 0.00267029  0.04       -0.00438387]
 [-0.00133514 -0.00438387  0.01      ]]


As we can see, the main diagonal hasn't changed, but the other covariances are different. This means that the algorithm has changed the eigenvalues of the correlation matrix.

Also functions to transform covariance matrix into correlation matrix and back are available:

In [4]:
# Transforming our covariance matrix to a correlation matrix
corr_matrix = risk_estimators.cov_to_corr(cov_matrix)

# Outputting the result
print('The correlation matrix is:')
print(corr_matrix)

# The standard deviation
std = np.array([0.1,0.2,0.1])

# And back to the covariance matrix
cov_matrix_again = risk_estimators.corr_to_cov(corr_matrix, std)

# Outputting the result
print('The covariance matrix calculated back is:')
print(cov_matrix_again)
print('Exactly the same as the original one:')
print(cov_matrix)


The correlation matrix is:
[[ 1.   0.1 -0.1]
 [ 0.1  1.  -0.3]
 [-0.1 -0.3  1. ]]
The covariance matrix calculated back is:
[[ 0.01   0.002 -0.001]
 [ 0.002  0.04  -0.006]
 [-0.001 -0.006  0.01 ]]
Exactly the same as the original one:
[[ 0.01   0.002 -0.001]
 [ 0.002  0.04  -0.006]
 [-0.001 -0.006  0.01 ]]


### NCO and CVO functions

The CVO solves the problem described in the "Problem Statement" part using the provided formula

$w^* = \frac{V^{-1}a}{a'V^{-1}a}$

The NCO algorithm is following these steps:

- Get the covariance matrix of the outcomes as an input (and the vector of means if the target is to maximize the Sharpe ratio).
- Transform the covariance matrix to the correlation matrix and calculate the distance matrix based on it.
- Cluster the covariance matrix into subsets of highly-correlated variables.
- Compute the optimal weights allocation for every cluster (using the CVO).
- Reduce the original covariance matrix to a reduced one - where each cluster is represented by a single variable.
- Compute the optimal weights allocation for the reduced covariance matrix (using the CVO).
- Compute the final allocations as a dot-product of the allocations between the clusters and inside the clusters.


This is how the NCO and the CVO can be used:

In [5]:
# A class that has the NCO and CVO functions
nco = ml.portfolio_optimization.NCO()

# Covariance matrix
cov_matrix = np.array([[0.01, 0.002, -0.001],
                       [0.002, 0.04, -0.006],
                       [-0.001, -0.006, 0.01]])

# Vector of ones (goal is minimizing the variance)
mu_vec = np.array([1, 1, 1]).reshape(-1, 1)

# Maximum number of clusters to use in the correlation matrix clustering
max_num_clusters = 2

# Finding the optimal weights using the NCO
w_nco = nco.opt_port_nco(cov_matrix, mu_vec, max_num_clusters=max_num_clusters)

# Outputting the result
print('The optimal weights with minimum variance using the NCO are:')
print(w_nco)

The optimal weights with minimum variance using the NCO are:
[[0.43875825]
 [0.09237016]
 [0.4688716 ]]


Now using the simple CVO algorithm:

In [6]:
# Finding the optimal weights using the CVO
w_cvo = nco.opt_port(cov_matrix, mu_vec)

# Outputting the result
print('The optimal weights with minimum variance using the CVO are:')
print(w_cvo)

The optimal weights with minimum variance using the CVO are:
[[0.37686939]
 [0.14257228]
 [0.48055833]]


The results are different, so the NCO has clustered the correlation matrix. 

### MCOS function

The Monte Carlo Optimization Selection (MCOS) algorithm calculates the NCO allocations and a simple optimal allocation for multiple simulated pairs of mean vector and the covariance matrix to determine the most robust method for weights allocations for a given pair of means vector and a covariance vector. Basically it will show if the NCO or the CVO method is more robust.

However, the MCOS may support other optimization methods and compare their robustness.

The steps of the MCOS algorithm are:
- Get the covariance matrix and the means vector of the outcomes as an input (along with the simulation parameters to use).
- Drawing the empirical covariance matrix and the empirical means vector based on the true ones.
- If the kde_bwidth parameter is given, the empirical covariance matrix is de-noised.
- Based on the min_var_portf parameter, either the minimum variance or the maximum Sharpe ratio is targeted in weights allocation.
- Simple optimal allocation is applied to the empirical data to obtain the weights allocation.
- NCO is applied to the empirical data to obtain the weights allocation.
- Based on the original covariance matrix and the means vector a true optimal allocation is calculated.
- For each weights estimation in a method, a standard deviation between the true weights and the obtained weights is calculated.
- The error associated with each method is calculated as the mean of the standard deviation across all estimations for the method.

This is how the MCOS can be used:

In [7]:
# Covariance matrix
cov_mat = np.array([[1, 0.1, 0.2, 0.3],
                    [0.1, 1, 0.1, 0.2],
                    [0.2, 0.1, 1, 0.1],
                    [0.3, 0.2, 0.1, 1]])

# Vector of means
mu_vec = np.array([0, 0.1, 0.2, 0.3])
cov_mat = np.array([[1, 0.1, 0.2, 0.3],
                    [0.1, 1, 0.1, 0.2],
                    [0.2, 0.1, 1, 0.1],
                    [0.3, 0.2, 0.1, 1]])

# Number of observations to use to obtain empirical covariance matrix and the means vector
num_obs = 100

# Number of simulations to do in the MCOS
num_sims = 2

# The bandwidth of the KDE kernel.
kde_bwidth = 0.25

# Flag if the goal is the minimum variance
min_var_portf = True

# Flag if we want to use the Ledoit-Wolf shrinkage procedure
lw_shrinkage = False

# For the same result output
np.random.seed(1)

# Finding the optimal weights for minimum variance
w_cvo, w_nco = nco.opt_port_mcos(mu_vec, cov_mat, num_obs, num_sims, kde_bwidth, min_var_portf, lw_shrinkage)

# Outputting the result
print('The optimal weights with minimum variance using the NCO from simulations are:')
print(w_nco)
print('The optimal weights with minimum variance using the CVO from simulations are:')
print(w_cvo)

The optimal weights with minimum variance using the NCO from simulations are:
          0         1         2         3
0  0.181431  0.326042  0.165669  0.326857
1  0.221613  0.394977  0.204978  0.178432
The optimal weights with minimum variance using the CVO from simulations are:
          0         1         2         3
0  0.165431  0.344039  0.168434  0.322096
1  0.202771  0.427418  0.212893  0.156919


And the outputs can be analyzed based on the mean of the standard deviation with the true weight allocations.

In [8]:
# Finding the errors in estimations
err_cvo, err_nco = nco.estim_errors_mcos(w_cvo, w_nco, mu_vec, cov_mat, min_var_portf)

# Outputting the result
print('The mean of the standard deviation with the true weight for NCO is:')
print(err_nco)
print('The mean of the standard deviation with the true weight for CVO is:')
print(err_cvo)

The mean of the standard deviation with the true weight for NCO is:
0.0524762395749268
The mean of the standard deviation with the true weight for CVO is:
0.05839900155706323


Even on this small example, we can see that the NCO has performed better than the CVO method (the error measure is smaller).

### Sample Data Generating function

We can create a random vector of means and a random covariance matrix to properly test the algorithms described above. 

These created covariance matrix and vector of means have the characteristics of securities inside being grouped in clusters. The clusters pose a high correlation and also have different correlations between clusters in order to test the NCO and the MCOS algorithms.

This is how the Sample Data Generating function can be used:

In [9]:
# Number of blocks to have in a modeled matrix
num_blocks = 2

# The size of each block in a modeled matrix
block_size = 2

# Correlation inside a block
block_corr = 0.3

# Correlation between the clusters
std = 0.3

# Finding the random vector of means and covariance matrix
mu_vec, cov_matrix = nco.form_true_matrix(num_blocks, block_size, block_corr, std)

# Outputting the result
print('The generated vector of means is:')
print(mu_vec)
print('The generated covariance matrix is:')
print(cov_matrix)

The generated vector of means is:
[[-0.00089985]
 [ 0.2608328 ]
 [ 0.45695781]
 [ 0.0015997 ]]
The generated covariance matrix is:
       0      3      2      1
0  0.090  0.000  0.000  0.027
3  0.000  0.090  0.027  0.000
2  0.000  0.027  0.090  0.000
1  0.027  0.000  0.000  0.090


These covariance vector and matrix of means can be used to test the NCO and the CVO methods.

### Conclusion

This notebook describes the Nested Clustered Optimization (NCO) algorithm, the Monte Carlo Optimization Selection (MCOS) algorithm alongside with the De-noising algorithm and other helping functions. Also it shows how these can be used on some real examples.

The algorithms and the descriptions are  were originally presented by _Marcos Lopez de Prado_ in the paper __A Robust Estimator of the Efficient Frontier__  [available here](https://papers.ssrn.com/abstract_id=3469961).

Key takeaways from the notebook:
- Convex optimization solutions in the problems of portfolio optimization tend to be unstable.
- The instability comes from two sources
  - Noise in the input variables
  - Signal structure that magnifies the estimation errors in the input variables
- The De-noising algorithm calculates the eigenvalues of the correlation matrix and eliminates those that are higher than the theoretically estimated ones, as they are caused by noise.
- The Convex Optimization Solution (CVO) can either find optimal allocations for maximum Sharpe ratio or minimum variance.
- The Nested Clustered Optimization (NCO) breaks the correlation function into clusters and applies CVO individually to each cluster as well as to the correlation matrix where each element represents the cluster.
- The Monte Carlo Optimization Selection (MCOS) allows runnin optimization methods (in our case CVO and NCO) on multiple sets of empirical covariation matrixes and empirical vectors of means. It can show which method is more robust.
- The Sample Data Generating function can be used to get a covariance matrix that has distinct clusters with correlations inside a cluster and different correlations between those clusters.